# How to Interpolate?

In [1]:
#dataframes
import pandas as pd
import h5py
# import rosbag_pandas

#math
import numpy as np
import math
pd.TimeSeries = pd.Series 

#interpolation scipy
import scipy
from scipy import signal
import scipy.interpolate as interpolate
from scipy import integrate

import multiprocessing as mp
import os
from tqdm import tqdm 

#gps
from geopy import distance
import geopandas as gpd
import utm
import datetime as dt

import rosbag_pandas
#plots
import pylab as plt
import matplotlib.pyplot as plt
import matplotlib.animation as animation  
import figurefirst
from figurefirst import FigureLayout,mpl_functions


### Using Rosbag Pandas to import Bag

### Load Data

In [54]:
folder_path = '../../data/BagsOctober/Run05/h5/'

file_list = [file for file in os.listdir(folder_path) if file.endswith('.h5')]
sorted_file_list = sorted(file_list)  # Sort the file list if needed
print(sorted_file_list)
dfs = []

for file_name in sorted_file_list:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_hdf(file_path)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

df = df.rename(columns={'index': 'master_time'})
time = df.master_time - df.master_time[0]
df.insert(1, 'time', time)


['OctoberDesert1020_0.h5', 'OctoberDesert1020_1.h5', 'OctoberDesert1020_2.h5', 'OctoberDesert1020_3.h5', 'OctoberDesert1020_4.h5']


## Interpolation

### Getting the GPS and WindSensor Time

In [57]:
df["trisonica_time"] = df._trisonica_header_stamp_secs + (df._trisonica_header_stamp_nsecs*1e-9)
df["gps_time"] = df._ublox_gps_fix_header_stamp_secs+(df._ublox_gps_fix_header_stamp_nsecs*1e-9)
df["gps_vel_time"] = df._ublox_gps_fix_velocity_header_stamp_secs+(df._ublox_gps_fix_velocity_header_stamp_nsecs*1e-9)
df["imu_time"] = df._imu_data_header_stamp_secs +(df._imu_data_header_stamp_nsecs*1e-9)

### Separating ODOR, WIND, GPS in dataframes

In [58]:
df_odor = pd.DataFrame({'odor_time': df.master_time,
                        'odor': df._analog_output_data,})
df_wind = pd.DataFrame({'wind_time': df.trisonica_time,'trisonica_U': df._trisonica_westeast,
                        'trisonica_V': df._trisonica_northsouth,'trisonica_pressure': df._trisonica_pressure,
                       'trisonica_temperature': df._trisonica_temperature})
df_gps = pd.DataFrame({'gps_time': df.gps_time,'lat': df._ublox_gps_fix_latitude,
                       'lon': df._ublox_gps_fix_longitude,'alt':df._ublox_gps_fix_altitude})
df_gps_vel = pd.DataFrame({'gps_vel_time':df.gps_vel_time,
                        'gps_linear_x':df._ublox_gps_fix_velocity_twist_twist_linear_x,
                      'gps_linear_y':df._ublox_gps_fix_velocity_twist_twist_linear_y,
                      'gps_linear_z':df._ublox_gps_fix_velocity_twist_twist_linear_z})
df_imu = pd.DataFrame({'imu_time': df.imu_time,'imu_angular_x': df._imu_data_angular_velocity_x,
                     'imu_angular_y': df._imu_data_angular_velocity_y, 
                     'imu_angular_z':df._imu_data_angular_velocity_z,
                      'imu_linear_acc_x':df._imu_data_linear_acceleration_x,
                      'imu_linear_acc_y':df._imu_data_linear_acceleration_y,
                      'imu_linear_acc_z':df._imu_data_linear_acceleration_z})

#### Creating time column for odor by dropping NaNs

In [59]:
df_odor = df_odor[df_odor.odor.notna()]

### Combined time from three sensors and arrange based on time

In [60]:
dfn = pd.concat([df_odor, df_wind, df_gps, df_gps_vel, df_imu], sort=True)
combined_time = np.hstack([df_odor.odor_time, df.trisonica_time ,df.gps_time, df.gps_vel_time, df.imu_time])
dfn['time'] = combined_time
dfn = dfn.sort_values(by=['time'])

### Find Master Time 

In [61]:
dt_odor = np.median(np.diff(dfn.odor_time.dropna().values))
print('dt Odor: ', dt_odor)
dt_wind = np.median(np.diff(dfn.wind_time.dropna().values))
print('dt Wind: ', dt_wind)
dt_gps = np.median(np.diff(dfn.gps_time.dropna().values))
print('dt Gps: ', dt_gps)
dt_gps_vel = np.median(np.diff(dfn.gps_vel_time.dropna().values))
print('dt Gps Vel: ', dt_gps_vel)
dt_imu = np.median(np.diff(dfn.imu_time.dropna().values))
print('dt Imu: ', dt_imu)
first_time = np.max([dfn.odor_time.min(), dfn.wind_time.min(), dfn.gps_vel_time.min(),
                     dfn.gps_time.min(), dfn.imu_time.min()])
last_time = np.min([dfn.odor_time.max(), dfn.wind_time.max(), dfn.gps_time.max(),
                    dfn.gps_vel_time.max(),dfn.imu_time.max()])
# print(first_time,last_time)
master_time = np.arange(first_time, last_time, dt_odor)

dt Odor:  0.005000114440917969
dt Wind:  0.02499699592590332
dt Gps:  0.20000004768371582
dt Gps Vel:  0.20000004768371582
dt Imu:  0.003999948501586914


### Create Interpolation function

In [62]:
# create interpolation function and inteprolate sensor 1
df_odor_1= dfn[['odor_time', 'odor']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_odor_1.odor_time, df_odor_1.odor)
interp_odor = func_interp_sensor_1(master_time)

df_wind_1= dfn[['wind_time', 'trisonica_U']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_1.wind_time, df_wind_1.trisonica_U)
interp_wind_U = func_interp_sensor_1(master_time)

df_wind_2= dfn[['wind_time', 'trisonica_V']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_2.wind_time, df_wind_2.trisonica_V)
interp_wind_V = func_interp_sensor_1(master_time)

df_wind_3= dfn[['wind_time', 'trisonica_pressure']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_3.wind_time, df_wind_3.trisonica_pressure)
interp_wind_p = func_interp_sensor_1(master_time)

df_wind_4= dfn[['wind_time', 'trisonica_temperature']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_4.wind_time, df_wind_4.trisonica_temperature)
interp_wind_t = func_interp_sensor_1(master_time)


df_gps_lat= dfn[['gps_time', 'lat']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_lat.gps_time, df_gps_lat.lat)
interp_gps_lat = func_interp_sensor_1(master_time)

df_gps_lon= dfn[['gps_time', 'lon']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_lon.gps_time, df_gps_lon.lon)
interp_gps_lon = func_interp_sensor_1(master_time)

df_gps_alt= dfn[['gps_time', 'alt']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_alt.gps_time, df_gps_alt.alt)
interp_gps_alt = func_interp_sensor_1(master_time)

df_gps_x= dfn[['gps_vel_time', 'gps_linear_x']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_x.gps_vel_time, df_gps_x.gps_linear_x)
interp_gps_x = func_interp_sensor_1(master_time)

df_gps_y= dfn[['gps_vel_time', 'gps_linear_y']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_y.gps_vel_time, df_gps_y.gps_linear_y)
interp_gps_y = func_interp_sensor_1(master_time)

df_gps_z= dfn[['gps_vel_time', 'gps_linear_z']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_z.gps_vel_time, df_gps_z.gps_linear_z)
interp_gps_z = func_interp_sensor_1(master_time)


df_imu_x= dfn[['imu_time', 'imu_angular_x']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_x.imu_time, df_imu_x.imu_angular_x)
interp_imu_x = func_interp_sensor_1(master_time)

df_imu_y= dfn[['imu_time', 'imu_angular_y']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_y.imu_time, df_imu_y.imu_angular_y)
interp_imu_y = func_interp_sensor_1(master_time)


df_imu_z= dfn[['imu_time', 'imu_angular_z']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_z.imu_time, df_imu_z.imu_angular_z)
interp_imu_z = func_interp_sensor_1(master_time)


df_imu_linear_x= dfn[['imu_time', 'imu_linear_acc_x']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_linear_x.imu_time, df_imu_linear_x.imu_linear_acc_x)
interp_imu_linear_acc_x = func_interp_sensor_1(master_time)

df_imu_linear_y= dfn[['imu_time', 'imu_linear_acc_y']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_linear_y.imu_time, df_imu_linear_y.imu_linear_acc_y)
interp_imu_linear_acc_y = func_interp_sensor_1(master_time)


df_imu_linear_z= dfn[['imu_time', 'imu_linear_acc_z']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_linear_z.imu_time, df_imu_linear_z.imu_linear_acc_z)
interp_imu_linear_acc_z = func_interp_sensor_1(master_time)





### New Interpolated Dataframe

In [63]:
odor = pd.DataFrame({'master_time': master_time,
                        'odor': interp_odor,
                        'trisonica_U': interp_wind_U,
                        'trisonica_V': interp_wind_V,
                        'lat': interp_gps_lat,
                        'lon': interp_gps_lon,
                        'alt':interp_gps_alt,
                        'gps_linear_x':interp_gps_x,
                        'gps_linear_y':interp_gps_y,
                        'gps_linear_z':interp_gps_z,
                        'trisonica_pressure':interp_wind_p,
                        'trisonica_temperature':interp_wind_t,
                        'imu_angular_x': interp_imu_x,
                        'imu_angular_y': interp_imu_y,
                        'imu_angular_z': interp_imu_z,
                        'imu_linear_acc_x':interp_imu_linear_acc_x,
                        'imu_linear_acc_y':interp_imu_linear_acc_y,
                        'imu_linear_acc_z':interp_imu_linear_acc_z})

# odor.to_hdf('/home/ecc/data/DesertDataOct/Run01/Run01_Interpolated.h5',key='df', mode='w')
# odor.to_hdf('~/Documents/Myfiles/DataAnalysis/data/TrajectoryInt.h5',key='df', mode='w')


In [64]:
# odor.to_hdf('~/Documents/Myfiles/DataAnalysis/data/Sprints/HighRes/Forest/ForestDataJuly_S1.h5',key='odor', mode='w')

## Interpolating Wind and Odor

In [65]:
#ubuntu
fname='~/DataAnalysis/data/DesertWind/20201020_172919_windy_b_data.hdf'
# fname='~/Documents/Myfiles/DataAnalysis/data/Sprints/Forest/WindStationA.hdf'
#mac 
# fname = '~/DataAnalysis/data/ForestWindData/202177_171552_windgps_data.hdf'
wind_df=pd.read_hdf(fname)
# wind_df.keys()
wind_df.reset_index(inplace=True, drop=True) 
wind =pd.DataFrame()
wind = wind_df.loc[(wind_df.time_epoch >= odor.master_time.iloc[0]) & 
                   (wind_df.time_epoch <= odor.master_time.iloc[-1])]
# df2.to_hdf('/home/ecc/data/DesertDataOct/Run01/Wind01.h5', key='df2', mode='w')

### Converting lat lon information to meters

In [66]:
from shapely.geometry import Point
odor['geometry'] = odor.apply(lambda row: Point(row.lon, row.lat), axis=1)

geodf = gpd.GeoDataFrame(odor)
geodf.crs= {'init': 'epsg:4326'}
# change the projection of geodf
geodf = geodf.to_crs("+proj=utm +zone=11 +ellps=WGS84 +datum=WGS84 +units=m +no_def")

geodf["x"] = geodf.geometry.apply(lambda row:row.x)
geodf["y"] = geodf.geometry.apply(lambda row:row.y)

#source location to meters
source = (40.79376709443614, -119.15757512132207) #desert
# source =(39.252441,-119.878037)
u =  utm.from_latlon(40.79376709443614, -119.15757512132207)
# u =  utm.from_latlon(39.252441, -119.878037)
xfromsrc= geodf.x-u[0]
geodf.insert(9,'xsrc',xfromsrc) 

yfromsrc = geodf.y-u[1]
geodf.insert(10,'ysrc', yfromsrc)

/home/gadfly/DataAnalysis/pyresearch/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/home/gadfly/DataAnalysis/pyresearch/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


### Interpolate

In [67]:
df_odor_loc=pd.DataFrame({'odor_time': geodf.master_time,'xsrc': geodf.xsrc, 'ysrc':geodf.ysrc, 
                          'odor':geodf.odor, 'lat':geodf.lat, 'lon':geodf.lon,'alt':geodf.alt,
                          'trisonica_U':geodf.trisonica_U, 'trisonica_V':geodf.trisonica_V,
                          'trisonica_temperature':geodf.trisonica_temperature,
                          'trisonica_pressure':geodf.trisonica_pressure,
                          'gps_linear_x':geodf.gps_linear_x,
                          'gps_linear_y':geodf.gps_linear_y,
                          'gps_linear_z':geodf.gps_linear_z,
                          'imu_angular_x': geodf.imu_angular_x,
                          'imu_angular_y': geodf.imu_angular_y,
                          'imu_angular_z': geodf.imu_angular_z,
                          'imu_linear_acc_x':geodf.imu_linear_acc_z,
                          'imu_linear_acc_y':geodf.imu_linear_acc_z,
                          'imu_linear_acc_z':geodf.imu_linear_acc_z
                         })
df_wind_loc=pd.DataFrame({'wind_time': wind.time_epoch,'U': wind.U,'V': wind.V, 'W':wind.W,
                          'D':wind.D , 'S2':wind.S2})


df_new = pd.concat([df_odor_loc,df_wind_loc], sort=True)
combined_time = np.hstack([df_odor_loc.odor_time, df_wind_loc.wind_time])

df_new['time'] = combined_time
df_new=df_new.sort_values(by=['time'])

dt_odor = np.median(np.diff(df_new.odor_time.dropna().values))
print('dt Odor: ', dt_odor)
dt_wind = np.median(np.diff(df_new.wind_time.dropna().values))
print('dt Wind: ', dt_wind)

first_time = np.max([df_new.odor_time.min(), df_new.wind_time.min()])
last_time = np.min([df_new.odor_time.max(), df_new.wind_time.max()])
############## SELECT THE RESOLUTION HERE ####################
master_time = np.arange(first_time, last_time, dt_odor)

# create interpolation function and inteprolate sensor 1
df_odor_1= df_new[['odor_time', 'xsrc']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_odor_1.odor_time, df_odor_1.xsrc)
interp_odor_x = func_interp_sensor_1(master_time)

df_odor_2= df_new[['odor_time', 'ysrc']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_odor_2.odor_time, df_odor_2.ysrc)
interp_odor_y = func_interp_sensor_1(master_time)

df_odor_la= df_new[['odor_time', 'lat']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_odor_la.odor_time, df_odor_la.lat)
interp_odor_la = func_interp_sensor_1(master_time)

df_odor_lo= df_new[['odor_time', 'lon']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_odor_lo.odor_time, df_odor_lo.lon)
interp_odor_lo = func_interp_sensor_1(master_time)

df_odor_alt= df_new[['odor_time', 'alt']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_odor_alt.odor_time, df_odor_alt.alt)
interp_odor_alt = func_interp_sensor_1(master_time)

df_odor_3= df_new[['odor_time', 'odor']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_odor_3.odor_time, df_odor_3.odor)
interp_odor = func_interp_sensor_1(master_time)

df_gps_x= df_new[['odor_time', 'gps_linear_x']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_x.odor_time, df_gps_x.gps_linear_x)
interp_gps_x = func_interp_sensor_1(master_time)

df_gps_y= df_new[['odor_time', 'gps_linear_y']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_y.odor_time, df_gps_y.gps_linear_y)
interp_gps_y = func_interp_sensor_1(master_time)

df_gps_z= df_new[['odor_time', 'gps_linear_z']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_gps_z.odor_time, df_gps_z.gps_linear_z)
interp_gps_z = func_interp_sensor_1(master_time)



df_wind_1x= df_new[['odor_time', 'trisonica_U']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_1x.odor_time, df_wind_1x.trisonica_U)
interp_wind_Ux = func_interp_sensor_1(master_time)

df_wind_2x= df_new[['odor_time', 'trisonica_V']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_2x.odor_time, df_wind_2x.trisonica_V)
interp_wind_Vx = func_interp_sensor_1(master_time)

df_wind_3x= df_new[['odor_time', 'trisonica_pressure']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_3x.odor_time, df_wind_3x.trisonica_pressure)
interp_wind_px = func_interp_sensor_1(master_time)

df_wind_4x= df_new[['odor_time', 'trisonica_temperature']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_4x.odor_time, df_wind_4x.trisonica_temperature)
interp_wind_tx = func_interp_sensor_1(master_time)

df_imu_x= df_new[['odor_time', 'imu_angular_x']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_x.odor_time, df_imu_x.imu_angular_x)
interp_imu_x = func_interp_sensor_1(master_time)

df_imu_y= df_new[['odor_time', 'imu_angular_y']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_y.odor_time, df_imu_y.imu_angular_y)
interp_imu_y = func_interp_sensor_1(master_time)


df_imu_z= df_new[['odor_time', 'imu_angular_z']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_z.odor_time, df_imu_z.imu_angular_z)
interp_imu_z = func_interp_sensor_1(master_time)


df_imu_linear_x= df_new[['odor_time', 'imu_linear_acc_x']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_linear_x.odor_time, df_imu_linear_x.imu_linear_acc_x)
interp_imu_linear_acc_x = func_interp_sensor_1(master_time)

df_imu_linear_y= df_new[['odor_time', 'imu_linear_acc_y']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_linear_y.odor_time, df_imu_linear_y.imu_linear_acc_y)
interp_imu_linear_acc_y = func_interp_sensor_1(master_time)


df_imu_linear_z= df_new[['odor_time', 'imu_linear_acc_z']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_imu_linear_z.odor_time, df_imu_linear_z.imu_linear_acc_z)
interp_imu_linear_acc_z = func_interp_sensor_1(master_time)




df_wind_1= df_new[['wind_time', 'U']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_1.wind_time, df_wind_1.U)
interp_wind_U = func_interp_sensor_1(master_time)

df_wind_2= df_new[['wind_time', 'V']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_2.wind_time, df_wind_2.V)
interp_wind_V = func_interp_sensor_1(master_time)

df_wind_W= df_new[['wind_time', 'W']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_W.wind_time, df_wind_W.W)
interp_wind_W = func_interp_sensor_1(master_time)


df_wind_3= df_new[['wind_time', 'D']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_3.wind_time, df_wind_3.D)
interp_wind_D = func_interp_sensor_1(master_time)

df_wind_4= df_new[['wind_time', 'S2']].dropna()
func_interp_sensor_1 = scipy.interpolate.interp1d(df_wind_4.wind_time, df_wind_4.S2)
interp_wind_S2 = func_interp_sensor_1(master_time)



windn = pd.DataFrame({'master_time': master_time,
                            'xsrc': interp_odor_x,
                            'ysrc': interp_odor_y,
                            'lat': interp_odor_la,
                            'lon': interp_odor_lo, 
                            'alt': interp_odor_alt,
                            'odor': interp_odor,
                            'trisonica_u': interp_wind_Ux,
                            'trisonica_v': interp_wind_Vx,
                            'trisonica_pressure':interp_wind_px,
                            'trisonica_temperature':interp_wind_tx,
                            'gps_linear_x': interp_gps_x,
                            'gps_linear_y': interp_gps_y,
                            'gps_linear_z': interp_gps_z,
                            'imu_angular_x': interp_imu_x,
                            'imu_angular_y': interp_imu_y,
                            'imu_angular_z': interp_imu_z,
                            'imu_linear_acc_x':interp_imu_linear_acc_x,
                            'imu_linear_acc_y':interp_imu_linear_acc_y,
                            'imu_linear_acc_z':interp_imu_linear_acc_z,
                      
                            'U': interp_wind_U,
                            'V': interp_wind_V,
                            'W': interp_wind_W,
                            'D':interp_wind_D,
                            'S2':interp_wind_S2})

# windn.to_hdf('~/Documents/Myfiles/DataAnalysis/data/Sprints/HighRes/Windy/Run04Run05HD.h5',key='windn',mode='w')

dt Odor:  0.005000114440917969
dt Wind:  0.10000276565551758


In [68]:
windn.to_hdf('~/DataAnalysis/data/BagsOctober/Run05/Run05.h5',key='windn',mode='w')

In [70]:
df1 = pd.read_hdf('~/DataAnalysis/data/BagsOctober/Run04/Run04.h5')
df2 = pd.read_hdf('~/DataAnalysis/data/BagsOctober/Run05/Run05.h5')                  

windy=pd.concat([df1, df2], ignore_index=True)
windy

,master_time,xsrc,ysrc,lat,lon,alt,odor,trisonica_u,trisonica_v,trisonica_pressure,...,imu_angular_y,imu_angular_z,imu_linear_acc_x,imu_linear_acc_y,imu_linear_acc_z,U,V,W,D,S2
0,1.603230e+09,1.369364,-0.688607,40.793761,-119.157559,1170.133031,0.636523,-1.458321,-0.329161,883.500000,...,-0.008513,0.004400,-9.935289,-9.935289,-9.935289,1.970000,1.320000,-0.470000,236.000000,2.370000
1,1.603230e+09,1.369859,-0.688619,40.793761,-119.157559,1170.132872,0.634477,-1.659034,-0.429517,883.500000,...,-0.007811,0.003390,-9.908950,-9.908950,-9.908950,1.964004,1.322998,-0.468501,235.850108,2.367002
2,1.603230e+09,1.370491,-0.688635,40.793761,-119.157559,1170.132698,0.636433,-1.842042,-0.520025,883.498755,...,-0.008031,-0.002500,-9.908627,-9.908627,-9.908627,1.958009,1.325996,-0.467002,235.700216,2.364004
3,1.603230e+09,1.371123,-0.688650,40.793761,-119.157559,1170.132523,0.635413,-1.832737,-0.503556,883.483989,...,-0.008361,-0.005069,-9.913431,-9.913431,-9.913431,1.952013,1.328994,-0.465503,235.550324,2.361006
4,1.603230e+09,1.371756,-0.688666,40.793761,-119.157559,1170.132348,0.635561,-1.748599,-0.445462,883.463961,...,-0.008798,-0.004161,-9.922390,-9.922390,-9.922390,1.946017,1.331991,-0.464004,235.400432,2.358009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594378,1.603233e+09,46.277696,-33.475118,40.793476,-119.157017,1170.845559,0.795207,-0.232485,-0.715029,882.799988,...,0.000105,0.014474,-8.730543,-8.730543,-8.730543,2.570000,4.062102,-0.037443,212.503792,4.804659
594379,1.603233e+09,46.277696,-33.475118,40.793476,-119.157017,1170.845459,0.795256,-0.240116,-0.699768,882.799988,...,0.000078,0.013542,-8.717485,-8.717485,-8.717485,2.570000,4.051602,-0.035943,212.603792,4.795659
594380,1.603233e+09,46.277696,-33.475118,40.793476,-119.157017,1170.845359,0.795684,-0.247746,-0.684507,882.799988,...,-0.000271,0.018631,-8.729113,-8.729113,-8.729113,2.570000,4.041102,-0.034443,212.703791,4.786659
594381,1.603233e+09,46.277696,-33.475118,40.793476,-119.157017,1170.845259,0.796335,-0.255377,-0.669246,882.799988,...,-0.000577,0.019365,-8.733929,-8.733929,-8.733929,2.570000,4.030602,-0.032943,212.803791,4.777659


In [75]:
dt = windy.master_time[1] - windy.master_time[0]
time = []
time.append(0)
for i in range(1,len(windy)):
    time.append(time[i-1]+dt)
    
windy['time'] = time

In [78]:
windy.to_hdf('~/DataAnalysis/data/BagsOctober/HWS.h5',key='windy',mode='w')